In [ ]:
import os
import pickle
from datetime import datetime, timedelta
import pandas as pd
from gnn_package import training, preprocessing
from private_uoapi import (
    LightsailWrapper,
    LSAuth,
    LSConfig,
    DateRangeParams,
    convert_to_dataframe,
)
from gnn_package import paths
from gnn_package.config import get_config

In [ ]:
raw_file_name = "test_data_1wk.pkl"
raw_dir = paths.RAW_TIMESERIES_DIR
raw_file_path = os.path.join(raw_dir, raw_file_name)
print("Currently loading data from: ", raw_file_name)

preprocessed_file_name = "data_loaders_test_1wk.pkl"
preprocessed_dir = paths.PREPROCESSED_TIMESERIES_DIR
preprocessed_file_path = os.path.join(preprocessed_dir, preprocessed_file_name)
print("Currently saving data to: ", preprocessed_file_name)

In [ ]:
config_path = (
    "/Users/administrator/Code/python/phd-project-gnn/gnn_package/config/config.yml"
)
config = get_config(config_path)

data_loaders = training.preprocess_data(
    data_file=raw_file_path,
    config=config,
)

# get processed data
if os.path.exists(preprocessed_file_path):
    with open(preprocessed_file_path, "rb") as f:
        data_loaders = pickle.load(f)

else:
    with open(preprocessed_file_path, "wb") as f:
        pickle.dump(data_loaders, f)

In [ ]:
data_loaders.keys()

In [ ]:
data_loaders["train_loader"]

In [ ]:
# Train the model
results = training.train_model(
    data_loaders=data_loaders,
)

In [ ]:
# Save the trained model
from torch import save

save(results["model"].state_dict(), f"stgnn_model_{raw_file_name.split('.')[0]}.pth")